# Training Deep Neural Networks

Last chapter we talked about neural network in genral, it was a shallow one where there are just few layers and maybe multiple nurons in these layers, beside of that we discuss the different keras API like sequential and functional model, talked about the tensorboard for visualization the difference runs, the callbacks to control the behavior of the models and fine tune the hyperparameters.

In this chapter we will go deeper to train the deep neural network, which can help us doing complex tasks like classification hundred of objects not like what we have binary or 10-class problem, what about voice recognition and other complex problem. But with these problem to be solved there are different parts we should take care about like **Vanshing and Exploding Problems**, **Overfitting**, **Large NN with small data and vise Verse**, **Labeling the data and training time** all of these problem we will see how to handle to train deep NN.

## Vanshing and Exploding Problems

"The backpropagation algorithm works by going from the output layer to the input layer, propagating the error gradient on the way, Unfortunately, gradients often get smaller and smaller as the algorithm progresses down to the lower layers. As a result, the Gradient Descent update leaves the lower layer connection weights virtually unchanged, and training never converges to a good solution. This is called the vanishing gradients problem. In some cases, the opposite can happen: the gradients can grow bigger and bigger, so many layers get insanely large weight updates and the algorithm diverges ", This is the exploding gradients proplem.

One of the solution for these **Vanishing and Exploding Problems** is to combine some of the activation functions with some method of weights in lizards as in the table below.

<img src="images/1.png">

By default, Keras uses Glorot initialization with a uniform distribution. You can change this to He initialization by setting kernel_initializer="he_uniform" or kernel_initializer="he_normal"

In [33]:
import os
import time
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import pandas as pd
from functools import partial
%matplotlib inline
%load_ext tensorboard
import warnings
warnings.filterwarnings('ignore')


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

## He initialization instead of defaul Glorot initialization

In [3]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu",  kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="relu",  kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.4191 - accuracy: 0.5981 - val_loss: 0.9511 - val_accuracy: 0.7122
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8333 - accuracy: 0.7411 - val_loss: 0.7321 - val_accuracy: 0.7704
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6975 - accuracy: 0.7747 - val_loss: 0.6474 - val_accuracy: 0.7900
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6320 - accuracy: 0.7937 - val_loss: 0.5966 - val_accuracy: 0.8066
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5902 - accuracy: 0.8046 - val_loss: 0.5634 - val_accuracy: 0.8164
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5610 - accuracy: 0.8124 - val_loss: 0.5384 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5387 - accuracy: 0.8187 - val_loss: 0.5228 - val_accuracy:

###  He initialization with a uniform distribution, but based on fan_avg rather than fan_in as in table

In [4]:
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu",  kernel_initializer=he_avg_init),
    keras.layers.Dense(100, activation="relu",  kernel_initializer=he_avg_init),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))


Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.2572 - accuracy: 0.6165 - val_loss: 0.8611 - val_accuracy: 0.7222
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.7797 - accuracy: 0.7437 - val_loss: 0.7018 - val_accuracy: 0.7762
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6714 - accuracy: 0.7810 - val_loss: 0.6267 - val_accuracy: 0.7988
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6116 - accuracy: 0.7994 - val_loss: 0.5796 - val_accuracy: 0.8126
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5720 - accuracy: 0.8104 - val_loss: 0.5478 - val_accuracy: 0.8228
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5433 - accuracy: 0.8186 - val_loss: 0.5267 - val_accuracy: 0.8286
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5217 - accuracy: 0.8239 - val_loss: 0.5071 - val_accuracy:

## Nonsaturating Activation Functions

Another slution of solving **Vanishing and Exploding Problems** is to use Non-saturated activation function, as usual we use **Sigmoid function** which saturated on when z is very large or very small, but there are another activation function have been appeared like Relu but it was lead to problem of **dying nurons** because of output 0 when z is less than 0, and to come over this problem the LeakyRelu and other variants of Relu have been introduced like Randomized Relu which introduce another hyper paramter to tune which is α, but this help us avoid the problem of dying nurons.

Also, another activation function is outperform all of the Relu and its variance is  exponential linear unit (ELU), but it takes more time to train and test, but it converage in less number of epochs.

<table>
<tr>
<th><img src="images/2.png"></th>
<th><img src="images/3.png"></th>

</tr>
</table>
<th><img src="images/4.png"></th>

## LeakyReLU instead of Relu

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(alpha=.2),
    keras.layers.Dense(100, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(alpha=.2),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.3303 - accuracy: 0.5848 - val_loss: 0.8938 - val_accuracy: 0.7060
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8048 - accuracy: 0.7343 - val_loss: 0.7195 - val_accuracy: 0.7630
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6872 - accuracy: 0.7744 - val_loss: 0.6369 - val_accuracy: 0.7872
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6257 - accuracy: 0.7939 - val_loss: 0.5922 - val_accuracy: 0.8042
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5857 - accuracy: 0.8064 - val_loss: 0.5585 - val_accuracy: 0.8158
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5568 - accuracy: 0.8136 - val_loss: 0.5335 - val_accuracy: 0.8250
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5351 - accuracy: 0.8207 - val_loss: 0.5145 - val_accuracy:

## Parametric Relu instead of Relu
Now alpha we have used with .2 is learned by the model as model paramter.

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer='he_normal'),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer='he_normal'),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.2938 - accuracy: 0.6276 - val_loss: 0.8954 - val_accuracy: 0.7186
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8065 - accuracy: 0.7365 - val_loss: 0.7200 - val_accuracy: 0.7622
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6906 - accuracy: 0.7690 - val_loss: 0.6446 - val_accuracy: 0.7826
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6291 - accuracy: 0.7911 - val_loss: 0.5942 - val_accuracy: 0.8016
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5889 - accuracy: 0.8046 - val_loss: 0.5628 - val_accuracy: 0.8134
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5595 - accuracy: 0.8128 - val_loss: 0.5369 - val_accuracy: 0.8242
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5373 - accuracy: 0.8197 - val_loss: 0.5186 - val_accuracy:

## Batch Normalization

The previous solution for **Vanishing and Exploding Problems** can significantly reduce the vanishing and exploding gradients problems at the beginning of training, but it doesn’t guarantee that they won’t come back during training. So a new technique called Batch Normalization (BN) have been introduced to address the vanishing and exploding gradients problems.
The technique consists of adding an operation in the model just before or after the activation function of each hidden layer.

## BatchNormalization but after activation function

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10,  activation="softmax")
])

model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_12 (Dense)             (None, 300)               235500    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_13 (Dense)             (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100)              

In [8]:
# Let’s look at the parameters of the first BN layer. Two are trainable (by backprop), and two are not:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [9]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.8745 - accuracy: 0.7056 - val_loss: 0.5874 - val_accuracy: 0.8038
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6043 - accuracy: 0.7915 - val_loss: 0.5053 - val_accuracy: 0.8266
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5437 - accuracy: 0.8104 - val_loss: 0.4707 - val_accuracy: 0.8382
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5112 - accuracy: 0.8208 - val_loss: 0.4486 - val_accuracy: 0.8426
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4918 - accuracy: 0.8261 - val_loss: 0.4344 - val_accuracy: 0.8478
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4739 - accuracy: 0.8352 - val_loss: 0.4214 - val_accuracy: 0.8496
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4634 - accuracy: 0.8379 - val_loss: 0.4112 - val_accuracy:

## BatchNormalization but before activation function

**sometimes it give a better result than using it after activation function**

- Also look at the result over 86 instead of 84 in other models without Normalization

In [10]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,  kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 1.0902 - accuracy: 0.6584 - val_loss: 0.6968 - val_accuracy: 0.7810
Epoch 2/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.7111 - accuracy: 0.7713 - val_loss: 0.5737 - val_accuracy: 0.8120
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6178 - accuracy: 0.7955 - val_loss: 0.5140 - val_accuracy: 0.8280
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5675 - accuracy: 0.8096 - val_loss: 0.4787 - val_accuracy: 0.8382
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5362 - accuracy: 0.8179 - val_loss: 0.4533 - val_accuracy: 0.8456
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5131 - accuracy: 0.8247 - val_loss: 0.4366 - val_accuracy: 0.8512
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4953 - accuracy: 0.8299 - val_loss: 0.4243 - val_accuracy:

## Lets do it with ELU

In [11]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,  kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.ELU(),
    keras.layers.Dense(100, kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.ELU(),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.9392 - accuracy: 0.6887 - val_loss: 0.6524 - val_accuracy: 0.7822
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6456 - accuracy: 0.7843 - val_loss: 0.5573 - val_accuracy: 0.8108
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5797 - accuracy: 0.8018 - val_loss: 0.5124 - val_accuracy: 0.8294
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5428 - accuracy: 0.8132 - val_loss: 0.4859 - val_accuracy: 0.8410
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5164 - accuracy: 0.8228 - val_loss: 0.4671 - val_accuracy: 0.8466
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5001 - accuracy: 0.8273 - val_loss: 0.4526 - val_accuracy: 0.8508
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4884 - accuracy: 0.8303 - val_loss: 0.4421 - val_accuracy:

## BatchNormalization after activation function with ELU

In [12]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,  kernel_initializer='he_normal'),
    keras.layers.ELU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, kernel_initializer='he_normal'),
    keras.layers.ELU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.8618 - accuracy: 0.7054 - val_loss: 0.5645 - val_accuracy: 0.8056
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5814 - accuracy: 0.8004 - val_loss: 0.4947 - val_accuracy: 0.8312
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5264 - accuracy: 0.8176 - val_loss: 0.4626 - val_accuracy: 0.8436
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4985 - accuracy: 0.8257 - val_loss: 0.4445 - val_accuracy: 0.8502
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4794 - accuracy: 0.8324 - val_loss: 0.4304 - val_accuracy: 0.8554
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4639 - accuracy: 0.8381 - val_loss: 0.4203 - val_accuracy: 0.8602
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4524 - accuracy: 0.8415 - val_loss: 0.4116 - val_accuracy:

## Change kernel_initializer 

In [13]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,  kernel_initializer='lecun_normal'),
    keras.layers.ELU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, kernel_initializer='lecun_normal'),
    keras.layers.ELU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7550 - accuracy: 0.7437 - val_loss: 0.5251 - val_accuracy: 0.8270
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5437 - accuracy: 0.8123 - val_loss: 0.4680 - val_accuracy: 0.8418
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4998 - accuracy: 0.8263 - val_loss: 0.4410 - val_accuracy: 0.8508
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4756 - accuracy: 0.8343 - val_loss: 0.4232 - val_accuracy: 0.8538
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4546 - accuracy: 0.8421 - val_loss: 0.4087 - val_accuracy: 0.8574
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4414 - accuracy: 0.8462 - val_loss: 0.4015 - val_accuracy: 0.8594
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4249 - accuracy: 0.8511 - val_loss: 0.3931 - val_accuracy:

# Reusing Pretrained Layers

We have talked about the **Vanishing and Exploding Problems**, and different techniques to solve these problem from different weights initialization with different activation function, and lastely talked about the Batch Normalization.

Now another problem is that maybe we do not have large training set, or we do not have the resource to build deep neural network from scratch, but there are some people who trained different architectures of Network on similar tasks and we can use their network with our new task which similar to the task they done. **This called Transfer Learning**.

<img src="images/5.png" width="500" height="500">


**The more similar the tasks are, the more layers you want to reuse (starting with the lower layers).**

Try freezing all the reused layers first so gradient descent won’t modify them, then train the model and see the result. Then try unfreezing one or two of the top hidden layers to let backpropagation tweak them and see if performance improves and so on.

The more training data you have, the more layers you can unfreeze. **It is also useful to reduce the learning rate when you unfreeze reused layers: this will avoid wrecking their fine-tuned weights**.


## Transfer Learning With Keras

We will try to train model on just 8 classes from the dataset, then use this trained model with the other two classes.

In [14]:
def split_dataset(X, y):
    # ~ not operation that Inverts all the bits
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6] # All other classes
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [15]:
print(X_train_A.shape)
print(X_train_B.shape)

# from 0 to 7 because other classes are shifted down
print(y_train_A[:30])

# Contain only 0 or 1 1 if shirt and 0 if sandals
print(y_train_B[:30])

(43986, 28, 28)
(200, 28, 28)
[4 0 5 7 7 7 4 4 3 4 0 1 6 3 4 3 2 6 5 3 4 5 1 3 4 2 0 6 7 1]
[1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 1. 0. 1. 1. 1. 1.]


In [16]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(8, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(1e-3),
             metrics=['accuracy'])
history = model.fit(X_train_A, y_train_A, epochs=10, validation_data=(X_valid_A, y_valid_A))

Epoch 1/10
1375/1375 [==============================] - 5s 3ms/step - loss: 0.5127 - accuracy: 0.8306 - val_loss: 0.3328 - val_accuracy: 0.8899
Epoch 2/10
1375/1375 [==============================] - 5s 3ms/step - loss: 0.3499 - accuracy: 0.8812 - val_loss: 0.2947 - val_accuracy: 0.9043
Epoch 3/10
1375/1375 [==============================] - 5s 3ms/step - loss: 0.3204 - accuracy: 0.8916 - val_loss: 0.2778 - val_accuracy: 0.9091
Epoch 4/10
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3035 - accuracy: 0.8973 - val_loss: 0.2683 - val_accuracy: 0.9113
Epoch 5/10
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2897 - accuracy: 0.9032 - val_loss: 0.2590 - val_accuracy: 0.9131
Epoch 6/10
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2811 - accuracy: 0.9056 - val_loss: 0.2530 - val_accuracy: 0.9153
Epoch 7/10
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2761 - accuracy: 0.9077 - val_loss: 0.2507 - val_accuracy:

In [17]:
model.save('trained_models/transfer_learning_8_class.h5')

# Clone model
First load the model, then clone it to be not affected by other model we transfer the model to.

In [18]:
model_A = keras.models.load_model('trained_models/transfer_learning_8_class.h5')
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [19]:
# All layers except the output layer as we moved from 8 classes to 2 classes
# Also the input shape for the two model is the same so there is no problem
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
# As we classify image to either shirt or sandals
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))

## Notes!

Now we have used all the layers from model A we have trained with 8 classes, and as we mentioned in transfer learning, the new task we are dealing with is classifying image into either shirt or sandals, which is close to task we have trained.

- The input is the same
- It's image classification task
- Just they different in the output from 8 classes to 2 classes so we do not use the output layer from model_A

But as all layers of model_A are trained and we use these layers with model_B_on_A, but the output layer we have add to model_B_on_A is initialized with random weights, so its good to first **freeze the used layer** then let the model train for small number of epoch to get some good weights for  the output layer, and **unfreeze the used layer maybe just the top ones** and train again but with **small learning rate** to not change reused layer weights away, then again you can **unfreeze more layers and so on**, but each time you **freeze or unfreeze you need to compile again**.


In [20]:
# Freeze all layer except output layer
for layer in model_B_on_A.layers[:-1]: 
    layer.trainable = False

# Compile the model
model_B_on_A.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                    metrics=['accuracy'])

# X_train_B is the binary data 
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=5, validation_data=(X_valid_B, y_valid_B))

Epoch 1/5
7/7 [==============================] - 1s 74ms/step - loss: 1.2411 - accuracy: 0.3150 - val_loss: 0.9500 - val_accuracy: 0.4452
Epoch 2/5
7/7 [==============================] - 0s 13ms/step - loss: 0.7545 - accuracy: 0.5600 - val_loss: 0.6304 - val_accuracy: 0.6501
Epoch 3/5
7/7 [==============================] - 0s 17ms/step - loss: 0.4967 - accuracy: 0.7500 - val_loss: 0.4699 - val_accuracy: 0.7850
Epoch 4/5
7/7 [==============================] - 0s 15ms/step - loss: 0.3668 - accuracy: 0.8600 - val_loss: 0.3827 - val_accuracy: 0.8469
Epoch 5/5
7/7 [==============================] - 0s 13ms/step - loss: 0.2932 - accuracy: 0.9100 - val_loss: 0.3250 - val_accuracy: 0.8844


In [21]:
# unFreeze all layer except output layer
for layer in model_B_on_A.layers[:-1]: 
    layer.trainable = True
    
# Compile the model again but with less learning rate
model_B_on_A.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-4),
                    metrics=['accuracy'])


# X_train_B is the binary data 
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=10, validation_data=(X_valid_B, y_valid_B))

Epoch 1/10
7/7 [==============================] - 1s 34ms/step - loss: 0.2581 - accuracy: 0.9100 - val_loss: 0.3137 - val_accuracy: 0.8905
Epoch 2/10
7/7 [==============================] - 0s 13ms/step - loss: 0.2464 - accuracy: 0.9150 - val_loss: 0.3046 - val_accuracy: 0.8945
Epoch 3/10
7/7 [==============================] - 0s 13ms/step - loss: 0.2257 - accuracy: 0.9400 - val_loss: 0.2960 - val_accuracy: 0.8976
Epoch 4/10
7/7 [==============================] - 0s 20ms/step - loss: 0.2117 - accuracy: 0.9300 - val_loss: 0.2893 - val_accuracy: 0.9016
Epoch 5/10
7/7 [==============================] - 0s 17ms/step - loss: 0.2015 - accuracy: 0.9400 - val_loss: 0.2834 - val_accuracy: 0.9057
Epoch 6/10
7/7 [==============================] - 0s 16ms/step - loss: 0.2195 - accuracy: 0.9350 - val_loss: 0.2769 - val_accuracy: 0.9057
Epoch 7/10
7/7 [==============================] - 0s 14ms/step - loss: 0.2222 - accuracy: 0.9300 - val_loss: 0.2724 - val_accuracy: 0.9077
Epoch 8/10
7/7 [===========

In [22]:
# freeze some layer except output layer 
for layer in model_B_on_A.layers[:-2]: 
    layer.trainable = False
    
# Compile the model again
model_B_on_A.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                    metrics=['accuracy'])

# fit again for another 10 epochs with some freeze for layers
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=10, validation_data=(X_valid_B, y_valid_B))

Epoch 1/10
7/7 [==============================] - 1s 38ms/step - loss: 0.1873 - accuracy: 0.9550 - val_loss: 0.2548 - val_accuracy: 0.9087
Epoch 2/10
7/7 [==============================] - 0s 12ms/step - loss: 0.2044 - accuracy: 0.9550 - val_loss: 0.2526 - val_accuracy: 0.9097
Epoch 3/10
7/7 [==============================] - 0s 12ms/step - loss: 0.1833 - accuracy: 0.9450 - val_loss: 0.2502 - val_accuracy: 0.9097
Epoch 4/10
7/7 [==============================] - 0s 11ms/step - loss: 0.1807 - accuracy: 0.9650 - val_loss: 0.2480 - val_accuracy: 0.9138
Epoch 5/10
7/7 [==============================] - 0s 17ms/step - loss: 0.2294 - accuracy: 0.9450 - val_loss: 0.2455 - val_accuracy: 0.9138
Epoch 6/10
7/7 [==============================] - 0s 15ms/step - loss: 0.1957 - accuracy: 0.9350 - val_loss: 0.2432 - val_accuracy: 0.9158
Epoch 7/10
7/7 [==============================] - 0s 14ms/step - loss: 0.1729 - accuracy: 0.9650 - val_loss: 0.2410 - val_accuracy: 0.9158
Epoch 8/10
7/7 [===========

In [23]:
# unfreeze all layers again
for layer in model_B_on_A.layers[:-2]: 
    layer.trainable = True
    
# Compile the model again
model_B_on_A.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-4),
                    metrics=['accuracy'])

# fit again for another 10 epochs with some freeze for layers
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=10, validation_data=(X_valid_B, y_valid_B))

Epoch 1/10
7/7 [==============================] - 1s 36ms/step - loss: 0.1651 - accuracy: 0.9550 - val_loss: 0.2317 - val_accuracy: 0.9209
Epoch 2/10
7/7 [==============================] - 0s 15ms/step - loss: 0.1791 - accuracy: 0.9600 - val_loss: 0.2275 - val_accuracy: 0.9189
Epoch 3/10
7/7 [==============================] - 0s 15ms/step - loss: 0.1521 - accuracy: 0.9700 - val_loss: 0.2251 - val_accuracy: 0.9178
Epoch 4/10
7/7 [==============================] - 0s 15ms/step - loss: 0.1617 - accuracy: 0.9650 - val_loss: 0.2216 - val_accuracy: 0.9239
Epoch 5/10
7/7 [==============================] - 0s 14ms/step - loss: 0.1723 - accuracy: 0.9500 - val_loss: 0.2188 - val_accuracy: 0.9260
Epoch 6/10
7/7 [==============================] - 0s 13ms/step - loss: 0.1549 - accuracy: 0.9700 - val_loss: 0.2161 - val_accuracy: 0.9260
Epoch 7/10
7/7 [==============================] - 0s 13ms/step - loss: 0.1565 - accuracy: 0.9700 - val_loss: 0.2137 - val_accuracy: 0.9260
Epoch 8/10
7/7 [===========

In [24]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.2011 - accuracy: 0.9330


[0.2011478692293167, 0.9330000281333923]

In [25]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(lr=1e-3, momentum=.9),
             metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5315 - accuracy: 0.8151 - val_loss: 0.4191 - val_accuracy: 0.8560
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4220 - accuracy: 0.8510 - val_loss: 0.3880 - val_accuracy: 0.8656
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3883 - accuracy: 0.8630 - val_loss: 0.3583 - val_accuracy: 0.8708
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3647 - accuracy: 0.8712 - val_loss: 0.3691 - val_accuracy: 0.8732
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3499 - accuracy: 0.8758 - val_loss: 0.3464 - val_accuracy: 0.8752
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3334 - accuracy: 0.8805 - val_loss: 0.3545 - val_accuracy: 0.8754
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3239 - accuracy: 0.8843 - val_loss: 0.3254 - val_accuracy:

In [26]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(lr=1e-3, momentum=.9,
                                     nesterov=True), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5215 - accuracy: 0.8176 - val_loss: 0.4008 - val_accuracy: 0.8618
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4164 - accuracy: 0.8530 - val_loss: 0.3958 - val_accuracy: 0.8612
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3853 - accuracy: 0.8625 - val_loss: 0.3724 - val_accuracy: 0.8674
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3639 - accuracy: 0.8707 - val_loss: 0.3531 - val_accuracy: 0.8764
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3458 - accuracy: 0.8773 - val_loss: 0.3481 - val_accuracy: 0.8748
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3321 - accuracy: 0.8799 - val_loss: 0.3316 - val_accuracy: 0.8808
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3204 - accuracy: 0.8852 - val_loss: 0.3392 - val_accuracy:

In [27]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.RMSprop(learning_rate=1e-3, rho=.9),
             metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4718 - accuracy: 0.8292 - val_loss: 0.4020 - val_accuracy: 0.8526
Epoch 2/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3617 - accuracy: 0.8672 - val_loss: 0.3387 - val_accuracy: 0.8764
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3258 - accuracy: 0.8805 - val_loss: 0.3504 - val_accuracy: 0.8724
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2989 - accuracy: 0.8917 - val_loss: 0.3615 - val_accuracy: 0.8722
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2817 - accuracy: 0.8968 - val_loss: 0.3220 - val_accuracy: 0.8868
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2678 - accuracy: 0.9019 - val_loss: 0.3158 - val_accuracy: 0.8880
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2551 - accuracy: 0.9063 - val_loss: 0.3032 - val_accuracy:

In [28]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3,
                                                                 beta_1=.9, beta_2=.999), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4745 - accuracy: 0.8292 - val_loss: 0.4025 - val_accuracy: 0.8536
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3667 - accuracy: 0.8660 - val_loss: 0.4270 - val_accuracy: 0.8434
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3315 - accuracy: 0.8779 - val_loss: 0.3235 - val_accuracy: 0.8860
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3034 - accuracy: 0.8888 - val_loss: 0.3212 - val_accuracy: 0.8870
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2844 - accuracy: 0.8929 - val_loss: 0.3196 - val_accuracy: 0.8880
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2661 - accuracy: 0.9014 - val_loss: 0.3509 - val_accuracy: 0.8704
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2535 - accuracy: 0.9053 - val_loss: 0.3203 - val_accuracy:

In [29]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-2, decay=1e-4),
             metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5306 - accuracy: 0.8156 - val_loss: 0.4244 - val_accuracy: 0.8512
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4248 - accuracy: 0.8479 - val_loss: 0.3895 - val_accuracy: 0.8650
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3931 - accuracy: 0.8610 - val_loss: 0.3760 - val_accuracy: 0.8674
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3748 - accuracy: 0.8674 - val_loss: 0.3622 - val_accuracy: 0.8728
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3595 - accuracy: 0.8721 - val_loss: 0.3539 - val_accuracy: 0.8742
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3480 - accuracy: 0.8767 - val_loss: 0.3580 - val_accuracy: 0.8756
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3370 - accuracy: 0.8806 - val_loss: 0.3586 - val_accuracy:

In [30]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * .1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(beta_1=.9, beta_2=.999), 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), 
                   callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5186 - accuracy: 0.8131 - val_loss: 0.4566 - val_accuracy: 0.8356
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4153 - accuracy: 0.8493 - val_loss: 0.4003 - val_accuracy: 0.8540
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3768 - accuracy: 0.8626 - val_loss: 0.3864 - val_accuracy: 0.8646
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3468 - accuracy: 0.8731 - val_loss: 0.3582 - val_accuracy: 0.8724
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3213 - accuracy: 0.8815 - val_loss: 0.3617 - val_accuracy: 0.8714
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3060 - accuracy: 0.8864 - val_loss: 0.3463 - val_accuracy: 0.8764
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2858 - accuracy: 0.8931 - val_loss: 0.3239 - val_accuracy:

In [31]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return .01
    elif epoch < 15:
        return .005
    else:
        return .001

    
    
lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(beta_1=.9, beta_2=.999), metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), 
                   callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5210 - accuracy: 0.8137 - val_loss: 0.4529 - val_accuracy: 0.8332
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4199 - accuracy: 0.8471 - val_loss: 0.3866 - val_accuracy: 0.8610
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3906 - accuracy: 0.8568 - val_loss: 0.3625 - val_accuracy: 0.8694
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3630 - accuracy: 0.8645 - val_loss: 0.4027 - val_accuracy: 0.8616
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3475 - accuracy: 0.8713 - val_loss: 0.3378 - val_accuracy: 0.8792
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2991 - accuracy: 0.8897 - val_loss: 0.3151 - val_accuracy: 0.8904
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2884 - accuracy: 0.8930 - val_loss: 0.3331 - val_accuracy:

In [32]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='lecun_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(beta_1=.9, beta_2=.999), 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), 
                   callbacks=[lr_scheduler])

Epoch 1/20
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4751 - accuracy: 0.8303 - val_loss: 0.4016 - val_accuracy: 0.8508
Epoch 2/20
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3669 - accuracy: 0.8655 - val_loss: 0.3891 - val_accuracy: 0.8648
Epoch 3/20
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3307 - accuracy: 0.8783 - val_loss: 0.3106 - val_accuracy: 0.8878
Epoch 4/20
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3025 - accuracy: 0.8867 - val_loss: 0.3408 - val_accuracy: 0.8782
Epoch 5/20
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2839 - accuracy: 0.8941 - val_loss: 0.3141 - val_accuracy: 0.8856
Epoch 6/20
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2668 - accuracy: 0.9007 - val_loss: 0.3425 - val_accuracy: 0.8768
Epoch 7/20
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2507 - accuracy: 0.9062 - val_loss: 0.2899 - val_accuracy:

In [36]:
exponential_decay_fn = exponential_decay(lr0=.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

RegularizedDense = partial(keras.layers.Dense,
                          activation='elu', kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(.01))


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    keras.layers.BatchNormalization(),
    RegularizedDense(100),
    keras.layers.BatchNormalization(),
    RegularizedDense(10, activation='softmax', kernel_initializer='glorot_uniform')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(beta_1=.9, beta_2=.999), 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), 
                   callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 1.4176 - accuracy: 0.7038 - val_loss: 1.9205 - val_accuracy: 0.4756
Epoch 2/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.9414 - accuracy: 0.7519 - val_loss: 1.1757 - val_accuracy: 0.6632
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.9179 - accuracy: 0.7670 - val_loss: 0.9820 - val_accuracy: 0.7480
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.8557 - accuracy: 0.7753 - val_loss: 0.9199 - val_accuracy: 0.7324
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.8112 - accuracy: 0.7802 - val_loss: 0.8625 - val_accuracy: 0.7752
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.7954 - accuracy: 0.7868 - val_loss: 0.9766 - val_accuracy: 0.7332
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.7670 - accuracy: 0.7919 - val_loss: 0.7834 - val_accuracy:

In [37]:
exponential_decay_fn = exponential_decay(lr0=.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

RegularizedDense = partial(keras.layers.Dense,
                          activation='elu', kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l1_l2(.01))


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    keras.layers.BatchNormalization(),
    RegularizedDense(100),
    keras.layers.BatchNormalization(),
    RegularizedDense(10, activation='softmax', kernel_initializer='glorot_uniform')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(beta_1=.9, beta_2=.999), 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), 
                   callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 6.5325 - accuracy: 0.6712 - val_loss: 5.1332 - val_accuracy: 0.6788
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 4.5529 - accuracy: 0.7028 - val_loss: 4.4765 - val_accuracy: 0.7202
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 4.1590 - accuracy: 0.7121 - val_loss: 4.4665 - val_accuracy: 0.5820
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 3.8190 - accuracy: 0.7187 - val_loss: 3.7893 - val_accuracy: 0.6750
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 3.5012 - accuracy: 0.7225 - val_loss: 3.4912 - val_accuracy: 0.7388
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 3.2389 - accuracy: 0.7259 - val_loss: 3.3381 - val_accuracy: 0.6878
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 3.0283 - accuracy: 0.7309 - val_loss: 2.9650 - val_accuracy:

In [39]:
exponential_decay_fn = exponential_decay(lr0=.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

RegularizedDense = partial(keras.layers.Dense,
                          activation='elu', kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(.05))


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    RegularizedDense(300),
    keras.layers.BatchNormalization(),
    RegularizedDense(100),
    keras.layers.BatchNormalization(),
    RegularizedDense(10, activation='softmax', kernel_initializer='glorot_uniform')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(beta_1=.9, beta_2=.999), 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), 
                   callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 2.0111 - accuracy: 0.6754 - val_loss: 1.2051 - val_accuracy: 0.7238
Epoch 2/10
1719/1719 [==============================] - 8s 4ms/step - loss: 1.0891 - accuracy: 0.7375 - val_loss: 0.9478 - val_accuracy: 0.7794
Epoch 3/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.9674 - accuracy: 0.7516 - val_loss: 0.9121 - val_accuracy: 0.7746
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.9188 - accuracy: 0.7581 - val_loss: 1.0972 - val_accuracy: 0.7120
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.8736 - accuracy: 0.7645 - val_loss: 0.9104 - val_accuracy: 0.7566
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.8252 - accuracy: 0.7715 - val_loss: 0.7692 - val_accuracy: 0.8122
Epoch 7/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.8019 - accuracy: 0.7731 - val_loss: 0.7513 - val_accuracy:

In [40]:
exponential_decay_fn = exponential_decay(lr0=.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

RegularizedDense = partial(keras.layers.Dense,
                          activation='elu', kernel_initializer='he_normal')


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=.2),
    keras.layers.BatchNormalization(),
    RegularizedDense(300),
    keras.layers.Dropout(rate=.2),
    keras.layers.BatchNormalization(),
    RegularizedDense(100),
    keras.layers.Dropout(rate=.2),
    keras.layers.BatchNormalization(),
    RegularizedDense(10, activation='softmax', kernel_initializer='glorot_uniform')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(beta_1=.9, beta_2=.999), 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), 
                   callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5728 - accuracy: 0.7918 - val_loss: 0.4370 - val_accuracy: 0.8358
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4838 - accuracy: 0.8229 - val_loss: 0.3795 - val_accuracy: 0.8516
Epoch 3/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4457 - accuracy: 0.8359 - val_loss: 0.3755 - val_accuracy: 0.8568
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4238 - accuracy: 0.8453 - val_loss: 0.3462 - val_accuracy: 0.8720
Epoch 5/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4049 - accuracy: 0.8508 - val_loss: 0.3197 - val_accuracy: 0.8802
Epoch 6/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.3875 - accuracy: 0.8571 - val_loss: 0.3214 - val_accuracy: 0.8808
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3752 - accuracy: 0.8618 - val_loss: 0.3040 - val_accuracy:

In [41]:
exponential_decay_fn = exponential_decay(lr0=.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

RegularizedDense = partial(keras.layers.Dense,
                          activation='elu', kernel_initializer='he_normal')


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=.2),
    RegularizedDense(300),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=.2),
    RegularizedDense(100),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=.2),
    RegularizedDense(10, activation='softmax', kernel_initializer='glorot_uniform')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(beta_1=.9, beta_2=.999), 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), 
                   callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5645 - accuracy: 0.7979 - val_loss: 0.4224 - val_accuracy: 0.8476
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4586 - accuracy: 0.8335 - val_loss: 0.3744 - val_accuracy: 0.8612
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4252 - accuracy: 0.8446 - val_loss: 0.3628 - val_accuracy: 0.8588
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3983 - accuracy: 0.8539 - val_loss: 0.3210 - val_accuracy: 0.8794
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3790 - accuracy: 0.8594 - val_loss: 0.3110 - val_accuracy: 0.8834
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3630 - accuracy: 0.8688 - val_loss: 0.3008 - val_accuracy: 0.8880
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3484 - accuracy: 0.8706 - val_loss: 0.2924 - val_accuracy:

In [42]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3157 - accuracy: 0.8834


[0.3157493472099304, 0.883400022983551]

In [43]:
exponential_decay_fn = exponential_decay(lr0=.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

RegularizedDense = partial(keras.layers.Dense,
                          activation='elu', kernel_initializer='he_normal')


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    RegularizedDense(300),
    keras.layers.BatchNormalization(),
    RegularizedDense(100),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=.2),
    RegularizedDense(10, activation='softmax', kernel_initializer='glorot_uniform')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(beta_1=.9, beta_2=.999), 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), 
                   callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5260 - accuracy: 0.8116 - val_loss: 0.3712 - val_accuracy: 0.8654
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4125 - accuracy: 0.8497 - val_loss: 0.3861 - val_accuracy: 0.8556
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3748 - accuracy: 0.8645 - val_loss: 0.3190 - val_accuracy: 0.8834
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3401 - accuracy: 0.8751 - val_loss: 0.3288 - val_accuracy: 0.8796
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3214 - accuracy: 0.8822 - val_loss: 0.3513 - val_accuracy: 0.8746
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2985 - accuracy: 0.8894 - val_loss: 0.3332 - val_accuracy: 0.8812
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2815 - accuracy: 0.8948 - val_loss: 0.3460 - val_accuracy:

In [44]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3217 - accuracy: 0.8920


[0.3216792941093445, 0.8920000195503235]